In [1]:
import sys
import os

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print("Path configurado para importar módulos de 'chemai':")
print(ROOT_DIR)


from proxy import configure_proxy
configure_proxy(ROOT_DIR)

Path configurado para importar módulos de 'chemai':
C:\Users\f0pi\git\viscosidade-ai
Proxy configurado.


In [7]:
import os
import torch
import optuna
import pytorch_lightning as pl

import numpy as np
import random

from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from transformers import AutoTokenizer, AutoModel
from peft import LoraConfig, get_peft_model

from chemai.loader import DipprDatasetLoader
from chemai.train import train_test_split
from chemai.model import ChemBERTModel
from chemai.datamodule import ChemBERTDataModule
from chemai.callbacks import BestModelExporter

from sklearn.preprocessing import StandardScaler

torch.set_float32_matmul_precision('medium')

In [3]:
MODEL_NAME = "DeepChem/ChemBERTa-77M-MTR"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME)

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
GLOBAL_SEED = 13
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
os.environ["PYTHONHASHSEED"] = str(GLOBAL_SEED)

torch.manual_seed(GLOBAL_SEED)
torch.cuda.manual_seed_all(GLOBAL_SEED)
pl.seed_everything(GLOBAL_SEED, workers=True)
torch.use_deterministic_algorithms(False)

Seed set to 13


In [5]:
data_loader = DipprDatasetLoader(data_dir='../data/nist_dippr_data')
data_loader.prepare()
pure = data_loader.get_pure()
mix = data_loader.get_mix()
pure_train, pure_dev = train_test_split(
    smiles1=pure['train']["MOL"].to_list(), 
    temperatures=pure['train']["T"].values, 
    y=pure['train']["logV"].values
)
pure_test = {
    'smiles': pure['test']["MOL"].to_list(),
    'temperatures': pure['test']["T"].values,
    'y':  pure['test']["logV"].values
}

mix_train, mix_dev = train_test_split(
    smiles1=mix['train']["MOL_1"].to_list(),
    smiles2=mix['train']["MOL_2"].to_list(),
    frac=mix['train']["MolFrac_1"].values,
    temperatures=mix['train']["T"].values,
    y=mix['train']["logV"].values,
)
mix_test = {
    "smiles_1": mix['test']["MOL_1"].to_list(),
    "smiles_2": mix['test']["MOL_2"].to_list(),
    "frac": mix['test']["MolFrac_1"].values,
    "temperatures": mix['test']["T"].values,
    "y": mix['test']["logV"].values,
}

print(f"Puro: train={len(pure_train['temperatures'])}, dev={len(pure_dev['temperatures'])}, test={len(pure_test['temperatures'])}")
print(f"Mix: train={len(mix_train['temperatures'])}, dev={len(mix_dev['temperatures'])}, test={len(mix_test['temperatures'])}")


Puro: train=5268, dev=1450, test=885
Mix: train=20635, dev=5254, test=5585


In [8]:
scaler_pure = StandardScaler()
pure_train['temperatures'] = scaler_pure.fit_transform(pure_train['temperatures'].reshape(-1, 1)).reshape(-1)
pure_dev['temperatures'] = scaler_pure.transform(pure_dev['temperatures'].reshape(-1, 1)).reshape(-1)
pure_test['temperatures'] = scaler_pure.transform(pure_test['temperatures'].reshape(-1, 1)).reshape(-1)

scaler_mix = StandardScaler()
mix_train['temperatures'] = scaler_mix.fit_transform(mix_train['temperatures'].reshape(-1, 1)).reshape(-1)
mix_dev['temperatures'] = scaler_mix.transform(mix_dev['temperatures'].reshape(-1, 1)).reshape(-1)
mix_test['temperatures'] = scaler_mix.transform(mix_test['temperatures'].reshape(-1, 1)).reshape(-1)

In [9]:
pure_dm = ChemBERTDataModule(tokenizer, pure_train, pure_dev, pure_test, batch_size=64, max_length=35)
mix_dm = ChemBERTDataModule(tokenizer, mix_train, mix_dev, mix_test, batch_size=64, max_length=25)


In [19]:
def objective(trial, mode, use_lora, tokenizer, base_model, datamodule):
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    lr_head = trial.suggest_float("lr_head", 1e-4, 5e-3, log=True)
    lr_lora = trial.suggest_float("lr_lora", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    if use_lora:
        lora_cfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, task_type="FEATURE_EXTRACTION")
        model_base = get_peft_model(base_model, lora_cfg)
    else:
        model_base = base_model
    model = ChemBERTModel(
        base_model=model_base,
        mode=mode,
        hidden_dim=hidden_dim,
        dropout=dropout,
        lr_head=lr_head,
        lr_lora=lr_lora,
        weight_decay=weight_decay,
    )
    ckpt = ModelCheckpoint(monitor="val_r2", save_top_k=1, mode="max")
    early = EarlyStopping(monitor="val_r2", patience=3, mode="max")
    trainer = pl.Trainer(max_epochs=50, 
                         accelerator="gpu" if torch.cuda.is_available() else "cpu", 
                         callbacks=[ckpt, early], 
                         enable_progress_bar=True)
    trainer.fit(model, datamodule)
    metrics = trainer.callback_metrics
    return metrics.get("val_r2", torch.tensor(10.0)).item()


In [20]:
def optuna_tune(mode, use_lora, tokenizer, base_model, datamodule):
    def optuna_objective(trial):
        return objective(trial, mode, use_lora, tokenizer, base_model, datamodule)
    sampler = optuna.samplers.TPESampler(seed=GLOBAL_SEED)
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(optuna_objective, n_trials=20)
    return study.best_params

In [21]:
def train_variant(mode, use_lora, datamodule, export_dir='../models/torch'):
    best_params = optuna_tune(mode, use_lora, tokenizer, base_model, datamodule)

    if use_lora:
        lora_cfg = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.1, task_type="FEATURE_EXTRACTION")
        model_base = get_peft_model(base_model, lora_cfg)
    else:
        model_base = base_model

    model = ChemBERTModel(
        base_model=model_base,
        mode=mode,
        hidden_dim=best_params['hidden_dim'],
        dropout=best_params['dropout'],
        lr_head=best_params['lr_head'],
        lr_lora=best_params['lr_lora'],
        weight_decay=best_params['weight_decay'],
    )

    ckpt = ModelCheckpoint(monitor="val_loss", save_top_k=1, mode="min")
    early = EarlyStopping(monitor="val_loss", patience=5, mode="min")
    exporter = BestModelExporter(export_dir=f"{export_dir}/{mode}_{'lora' if use_lora else 'base'}", tokenizer=tokenizer)

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        callbacks=[ckpt, early, exporter],
    )

    trainer.fit(model, datamodule)
    test_metrics = trainer.test(model, datamodule)

    if test_metrics and isinstance(test_metrics, list):
        m = test_metrics[0]
        test_loss = m.get("test_loss", None)
        test_r2 = m.get("test_r2", None)
        print(f"[{mode} - {'LoRA' if use_lora else 'Base'}] Test Loss: {test_loss:.6f} | Test R²: {test_r2:.6f}")


In [22]:
train_variant('pure', False, pure_dm)
train_variant('pure', True, pure_dm)
train_variant('mix', False, mix_dm)
train_variant('mix', True, mix_dm)

[I 2025-12-15 21:54:15,745] A new study created in memory with name: no-name-59f51039-7ab0-42ea-8330-97e3384b1e6a
GPU available: True (cuda), used: True


🧊 Base model congelado (train apenas MLP).


TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type         ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ base_model │ RobertaModel │  3.4 M │ eval  │     0 │
│ 1 │ mlp        │ Sequential   │ 99.1 K │ train │     0 │
│ 2 │ train_r2   │ R2Score      │      0 │ train │     0 │
│ 3 │ val_r2     │ R2Score      │      0 │ train │     0 │
└───┴────────────┴──────────────┴────────┴───────┴───────┘

Trainable params: 99.1 K                                                                                           
Non-trainable params: 3.4 M                                                                                        
Total params: 3.5 M                                                                                                
Total estimated model params size (MB): 14                                                                         
Modules in train mode: 7                                                                                           
Modules in eval mode: 66                                                                                           
Total FLOPs: 0

...

🧊 Base model congelado (train apenas MLP).


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type         ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ base_model │ RobertaModel │  3.4 M │ eval  │     0 │
│ 1 │ mlp        │ Sequential   │ 24.8 K │ train │     0 │
│ 2 │ train_r2   │ R2Score      │      0 │ train │     0 │
│ 3 │ val_r2     │ R2Score      │      0 │ train │     0 │
└───┴────────────┴──────────────┴────────┴───────┴───────┘

Trainable params: 24.8 K                                                                                           
Non-trainable params: 3.4 M                                                                                        
Total params: 3.5 M                                                                                                
Total estimated model params size (MB): 13                                                                         
Modules in train mode: 7                                                                                           
Modules in eval mode: 66                                                                                           
Total FLOPs: 0

Output()

In [23]:
print('abc')

abc
